# Occupancy Forecasting Notebook

In [1]:
#Imports
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pandas.tseries.offsets import DateOffset

from datetime import datetime
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots


from sklearn.model_selection import train_test_split, KFold

from _dfguru import DataFrameGuru as DFG
from _preprocessing import PLCount

# 1.) Data Preprocessing

In [2]:
#Reading the dataset
dfg = DFG()

data = dfg.read_data(
    path_repo="../data/cleaned_data", 
    file_name="frequency_data", 
)

data = dfg.clean_signal_data(data)

min_timestamp = data["datetime"].min().replace(hour=0, minute=0, second=0, microsecond=0)
max_timestamp = data["datetime"].max().replace(hour=0, minute=0, second=0, microsecond=0) + DateOffset(days=1)

In [3]:

course_dates_data = dfg.filter_by_timestamp(
    dataframe = dfg.read_data(
        path_repo="../data/cleaned_data", 
        file_name="course_dates", 
    ),
    start_time = min_timestamp,
    end_time = max_timestamp,
    time_column = "start_time"
)


course_info_data = dfg.read_data(
    path_repo="../data/cleaned_data", 
    file_name="course_info", 
)


### 1.1) Filter out Facility Management Events

Maybe a course is missing: 8.4. - 28.6.

## Check again after preprocessing is final

HS18:
* 22.4 17:15-18:00 - safe something missing
* 26.4. 8:30-10:00 - safe something missing
* 3.5. 12:00 bis 18:30 - there is something happening -> plot but it doesn't seem to be a lecture
* 15.5. 15:15 bis 17:45 - there is also something happening 
* 4.6. 17:00 - 19:00 -> There is also something happening
* 20.6. 15:30 - 17:00, 17:15-18:45

HS19:
* 12.4. 13:45 - 15:45 -> no normal lecture is something different
* 13.5. 13:45 - 15:15, afterwards - 19:05 (something different)  -> check preprocessing! -> 5 events ruined that afternoon
* 15.5. 15:30 - 17:30 -> safe a lecture
* 5.6. 8:30 - 10:00 -> student preparing for exam
* 26.6. 8:30 - 10:00, 12:00-14:30 -> something happening!

#### Try to clean events outside lecture hours -> show that sometimes Kusss seems to be incorrect:


Visually proof that something might be off! At the times above

In [4]:
#course_dates_data = handler.derive_day(course_dates_data, "start_time")
#course_dates_data_reduced = handler.filter_by_date_range(course_dates_data, "start_time", min_date, max_date)

#lecture_hours = []
#for grouping, df in course_dates_data_reduced.groupby(["room_id", "day"]):
    
#    max_time = (df["end_time"].max() + DateOffset(minutes=15)).time()
#    min_time = (df["start_time"].min() - DateOffset(minutes=15)).time()

#    group_mask = (data["room_id"] == grouping[0]) & (data["datetime"].dt.date == grouping[1])
#    data_group = data[group_mask]
    
#    #if (df["start_time"].min().date() == datetime.date(datetime(2024, 5, 15))) and grouping[0] == 0:
#    #    print(grouping)
#    #    print(df.sort_values(by="start_time").to_string())
#    #    print(min_time, max_time)
#    #    print(data_group.to_string())
#    #    print("-------------------")
#    #    print()
        
#    group_mask = (data["room_id"] == grouping[0]) & (data["datetime"].dt.date == grouping[1])
#    data_group = data[group_mask]
    
    
#    outside_hours_mask = (data_group["datetime"].dt.time < min_time) | (data_group["datetime"].dt.time > max_time)
#    #print(data_group[outside_hours_mask].to_string())
#    #lecture_hours = group_mask & (data["datetime"].dt.time >= min_time) & (data["datetime"].dt.time <= max_time)

## 1. Data Cleaning with PLCount

In [5]:

frequency = "15min"
plcount = PLCount()

data_dict = {}
for room_id, df in data.groupby("room_id"):
    data_filterd_room = dfg.filter_by_roomid(data, room_id)
    occ_list = plcount.run_on_whole_dataset(data_filterd_room, dfg, frequency)
    data_dict[room_id] = pd.concat(occ_list).drop_duplicates(subset="datetime")

  0%|          | 0/82 [00:00<?, ?it/s]

100%|██████████| 82/82 [00:00<00:00, 87.92it/s]


In [6]:
#occupancy_data_cleaned = pd.concat(occ_list).drop_duplicates(subset="datetime")
#occupancy_data_raw = dfg.calc_occupancy_count(data_filterd_room, "datetime", frequency)

### 1.1 Plot differences

In [7]:
## raw_data
#fig = go.Figure()

#fig.add_trace(
#    go.Scatter(
#        x=occupancy_data_raw["datetime"], 
#        y=occupancy_data_raw["CC"],
#        mode="lines",
#        name="Raw Occupancy"))

#fig.add_trace(
#    go.Scatter(
#        x=occupancy_data_cleaned["datetime"], 
#        y=occupancy_data_cleaned["CC"],
#        mode="lines",
#        name="Reset at Midnight"))

#fig.add_trace(
#    go.Scatter(
#        x=occupancy_data_cleaned["datetime"], 
#        y=occupancy_data_cleaned["CC_estimates"],
#        mode="lines",
#        name="PLCount Estimates"))

#fig.show()

## 2. Contextual features

Classic Contextual features: (mostly endogenous)
* Sangogboye, Arendt: Dayname, Daytype(weekend/weekday), Season(Winter, Summer), Holiday(yes/no)(exogenous)
* Sangogboye, Kjaaergard: Dayname, Daytype, Season, Holiday
* Araz Ashouri, Member, IEEE, Guy R. Newsham: Hour of Day (Sin + Cosine), Day of week (0 = Holiday, Weekends; 1 = Weekday) 

My ideas:
* Include Zwickeltag - exogenous
* Make use of lecture schedule from Kusss - exogenous



In [15]:
dates = course_dates_data.sort_values(by="start_time").reset_index(drop=True)

for room_id in data_dict:
    
    occ_df = data_dict[room_id]
    occ_df["course_number"] = 0
    occ_df["exam"] = 0
    
    dates_filtered = dfg.filter_by_roomid(dates, room_id)
    
    for grouping, row in dates_filtered.groupby(["start_time", "end_time"]):
        # translate to entry in time series
        time_mask = (occ_df["datetime"] >= grouping[0]) & (occ_df["datetime"] <= grouping[1])
        occ_df.loc[time_mask, "course_number"] = ",".join(row["course_number"].values)
        occ_df.loc[time_mask, "exam"] = row["exam"].values[0]
        
    
    occ_df["lecture"] = occ_df["course_number"].apply(lambda x: -25 if x != 0 else 0)
    occ_df["exam"] = occ_df["exam"].apply(lambda x: -25 if x != 0 else 0)

/tmp/ipykernel_2851018/70990217.py:14: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '258.410' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

/tmp/ipykernel_2851018/70990217.py:15: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'False' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

/tmp/ipykernel_2851018/70990217.py:14: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '238.302' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.

/tmp/ipykernel_2851018/70990217.py:15: FutureWarning:

Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'False' has dtype incompatible with 

In [16]:
# raw_data
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=occ_df["datetime"], 
        y=occ_df["lecture"],
        mode="lines",
        name="Course Boolean"))

fig.add_trace(
    go.Scatter(
        x=occ_df["datetime"], 
        y=occ_df["exam"],
        mode="lines",
        name="Exam Boolean"))

fig.add_trace(
    go.Scatter(
        x=occ_df["datetime"], 
        y=occ_df["CC_estimates"],
        mode="lines",
        name="PLCount Estimates"))

fig.show()

In [12]:
course_info_data.head(5)

,course_number,course_name,type,kind,lecturer,semester,ects,max_students,registered_students,no_dates_total,...,level,study_area,university,curriculum,assessment_criteria,teaching_methods,language,other_information,study_subject,room_id
0,340.100,Algorithmen und Datenstrukturen 1,VO,Präsenz,Alois Ferscha,2024S,"3,00",200,194,14.0,...,B1 - Bachelor 1. Jahr,Informatik,Johannes Kepler Universität Linz,Bachelorstudium Informatik,Klausur am Semesterende,Foliengestützter Vortrag,Deutsch,https://www.pervasive.jku.at/Teaching/,Software,0
1,531.163,Analysis II,UE,Präsenz,Jan-Michael Holzinger,2024S,"2,00",25,10,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Not available,0
2,531.160,Analysis II,VO,Präsenz,Friedrich Pillichshammer,2024S,"3,00",100,98,29.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Not available,0
3,256.906,Anwendung ERP-Systeme II,KS,Präsenz,N. N.,2024S,"3,00",60,59,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Not available,0
4,239.141,Behavioral Economics,KS,Präsenz,Matthias Fahn,2024S,"3,00",200,145,10.0,...,B2 - Bachelor 2. Jahr,Volkswirtschaftslehre,Johannes Kepler Universität Linz,Bachelorstudium Wirtschaftswissenschaften,Klausur,"Vortrag und Diskussion, Experimente",Englisch,NaN,Wahlfächer,0


## Validation Strategy and Train-Test Split

## 5. Basic Statistics and Plots

In [ ]:
#data_filterd = filter_room_id(data, 0)

Bar Charts

In [ ]:
# Paper: 
# Basic bar plots count of events after resampling
# Check if it follows some distribution

#data_filterd = filter_by_event_type(data_filterd, 1)
#data_resampled = resample(data_filterd, "datetime", "5min", "count")


#data_trimmed = data_resampled[data_resampled["event_type"] > 0]
#value_counts = data_trimmed.value_counts("event_type")

#fig = go.Figure()
## line plot
#fig.add_trace(
#    go.Bar(
#        x=value_counts.index, 
#        y=value_counts, 
#        name='value_counts'))


Boxplot

In [13]:
# Boxplot plots -> Boxplot for every day and time
# Do for different events and rooms

#data_plot = occupancy_data_cleaned
#data_plot = handler.derive_time(data_plot)
#data_plot = handler.derive_weekday(data_plot)
#data_plot = handler.derive_week(data_plot)
#data_plot = data_plot.drop("datetime", axis=1)

#data_plot = data_plot.groupby(["week", "time", "weekday"]).sum().reset_index()

#print(data_plot)
#fig = px.box(data_plot, x="time", y="CC_estimates")
## make figure larger
#fig.update_layout(
#    autosize=False,
#    width=1000,
#    height=500,
#)
#fig.show()

More plots in the other notebook

## 1.Patterns

clustering -> different algorithms, k-means

Papers tried: 

### 1.1 Occupancy data analytics and prediction: A case study - Xin Liang a, b, Tianzhen Hong b, *, Geoffrey Qiping Shen

Distance Measures: Eucleadian, Dynamic Time Warp, Correlation Simmilarity -> compared among all different configurations

Clustering Algos: K-Means, 

Algorithm Evaluation: Davies-Bouldin index, to find best k

Insights of the paper:

* High variability in the data statistical methods using some kind of mean will fail miserably!

In [ ]:
import json
from collections import defaultdict
import itertools
from tqdm import tqdm


In [ ]:
def filter_by_time(data, time_column, start_time, end_time):
    
    return data[(data[time_column].dt.time >= start_time) & (data[time_column].dt.time <= end_time)]

def plot_cluster_details(cluster_number, clustering_model, samples, time_axis, plot_centers):
    
    indices = np.arange(len(samples))
    
    fig = go.Figure()
    
    cluster_indices = indices[clustering_model.labels_ == cluster_number]
    
    if plot_centers:
        cluster_center = clustering_model.cluster_centers_[cluster_number]
        fig.add_trace(
            go.Scatter(
                x=time_axis, 
                y=cluster_center, 
                name=f'Center',
                mode="lines+markers"        
            )
        )

    
    print(f"################ {cluster_number} ################")
    print(cluster_indices)

    for x in range(len(cluster_indices)):
        fig.add_trace(
            go.Scatter(
                x=time_axis, 
                y=samples[cluster_indices[x]], 
                mode="lines+markers"        
            )
        )
    
    return fig


In [ ]:
# split into daily samples
data_clustering = occupancy_data_cleaned
data_clustering = dfg.derive_time(data_clustering)
data_clustering = dfg.derive_weekday(data_clustering)
data_clustering = dfg.derive_week(data_clustering)
data_clustering = dfg.derive_day(data_clustering)

occupancy_counts = []
sample_info = []
group_list = []
times = data_clustering["time"].unique()

for group in data_clustering.groupby("day"):
    
    occupancy_counts.append(group[1]["CC_estimates"].values)
    sample_info.append(group[1][["weekday", "weekday_name", "week", "day"]].values[0])
    group_list.append(group[0])

occupancy_counts = np.array(occupancy_counts[:-1])
sample_info = np.array(sample_info[:-1])
group_list = np.array(group_list[:-1])


In [ ]:
occupancy_counts

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
from sklearn.cluster import KMeans, AffinityPropagation, MeanShift, SpectralClustering, AgglomerativeClustering
from sklearn.metrics import davies_bouldin_score, pairwise_distances

from scipy.spatial.distance import pdist, squareform



In [ ]:
from pyclustering.cluster.kmeans import kmeans
from pyclustering.cluster.kmedians import kmedians
from pyclustering.cluster.center_initializer import kmeans_plusplus_initializer

from pyclustering.utils.metric import distance_metric, type_metric
from scipy.spatial.distance import correlation, cosine
from dtaidistance import dtw
from tslearn.metrics import dtw as dtw_tslearn
import heapq

In [ ]:
room_capacity = handler.filter_by_room_id(course_dates_data, 0)["room_capacity"].unique()

In [ ]:
import warnings
import random
np.warnings = warnings

# list of 50 random seeds between 0 and 100
seeds = list(range(0,1000))
seeds = random.sample(seeds, 100)

cluster_data = occupancy_counts

parameters = {"n_clusters":list(range(2,10)),
              "distance_metric":[
                  distance_metric(type_metric.EUCLIDEAN),
                  distance_metric(type_metric.CHEBYSHEV),
                  distance_metric(type_metric.MANHATTAN),
                  distance_metric(type_metric.EUCLIDEAN_SQUARE),
                  distance_metric(type_metric.USER_DEFINED, func=dtw),
                  distance_metric(type_metric.USER_DEFINED, func=dtw_tslearn),
                  distance_metric(type_metric.USER_DEFINED, func=correlation),
                  distance_metric(type_metric.USER_DEFINED, func=cosine),                   
              ]}

# change -> first iterate over parameters than compute avg or median score for each parameter combination over all seeds and variance

comb_iterator = ParameterSearch(parameter_dict=parameters).grid_search_iterator(tqdm_bar=True)

combinations = []
avg_scores = []
median_scores = []
std_scores = []

avg_sizes = []
for i, params in enumerate(comb_iterator):
    
    bdi_scores = []
    sizes = []
    for seed in seeds:
    
        # sklearn
        #clustering_model = KMeans(n_clusters=params["n_clusters"], random_state=seed)
        #clustering_model.fit(occupancy_counts)
        #cluster_labels = clustering_model.labels_
        
        # pyclustering
        initial_centers = kmeans_plusplus_initializer(cluster_data, params["n_clusters"]).initialize()
        #clustering_model = kmeans(cluster_data, initial_centers, distance_metric=params["distance_metric"])
        clustering_model = kmedians(cluster_data, initial_centers, distance_metric=params["distance_metric"])
        clustering_model.process()
        
        cluster_labels = np.empty(len(occupancy_counts))
        for i, cluster in enumerate(clustering_model.get_clusters()):
            cluster_labels[cluster] = i

        cluster_sizes = np.sort([len(cluster) for cluster in clustering_model.get_clusters()])
        sizes.append(cluster_sizes)
        
        score = davies_bouldin_score(occupancy_counts, cluster_labels)
        bdi_scores.append(np.round(score, 8))
        
    combinations.append((i, params))
    avg_sizes.append(np.mean(sizes, axis=0))
    print(np.mean(sizes, axis=0))

    avg_scores.append(np.mean(bdi_scores))
    median_scores.append(np.median(bdi_scores))
    std_scores.append(np.var(bdi_scores))
    

  2%|▏         | 1/64 [00:01<01:49,  1.74s/it]

[13.89 68.11]


  3%|▎         | 2/64 [00:03<02:03,  2.00s/it]

[14.19 67.81]


  5%|▍         | 3/64 [00:05<01:55,  1.90s/it]

...

[ 1.    1.03  1.31  2.05  4.23  7.46 10.05 19.52 35.35]


 98%|█████████▊| 63/64 [05:25<00:05,  5.65s/it]

[ 1.    1.02  1.38  2.18  4.28  7.98 10.35 18.61 35.2 ]


100%|██████████| 64/64 [05:31<00:00,  5.18s/it]

[ 1.    1.06  1.44  2.21  4.46  7.55 10.35 18.51 35.42]


In [ ]:
np.array(sizes)[:,0].mean()

15.48

In [ ]:
np.mean(sizes, axis=0)

array([15.48, 66.52])

In [ ]:
k = 10
bot_k = np.argsort(avg_scores)[:k]
bot_k_std= np.array(std_scores)[bot_k]
final_choice = bot_k[np.argmin(bot_k_std)]

final_choice

61

In [ ]:
def plot_cluster_centers(n_clusters, cluster_centers, times):
    
    fig = go.Figure()
    
    for i in range(n_clusters):
        # line plot
        fig.add_trace(
            go.Scatter(
                x=times, 
                y=cluster_centers[i], 
                name=f'cluster {i}',
                mode="lines+markers"        
            )
        )
        
    return fig

In [ ]:
# test optimal configuration
optimal_params = combinations[61][1]
print(optimal_params, optimal_params["distance_metric"].get_type(), optimal_params["distance_metric"].get_function())

# sklearn
#clustering_model = KMeans(**optimal_params)
#clustering_model.fit(cluster_data)
#n_clusters = len(clustering_model.cluster_centers_)

initial_centers = kmeans_plusplus_initializer(cluster_data, optimal_params["n_clusters"]).initialize()
#clustering_model = kmeans(cluster_data, initial_centers, distance_metric=optimal_params["distance_metric"])
clustering_model = kmedians(cluster_data, initial_centers, distance_metric=params["distance_metric"])
clustering_model.process()

cluster_indices = clustering_model.get_clusters()
n_clusters = len(cluster_indices)
cluster_labels = np.empty(len(cluster_data))
for i, cluster in enumerate(cluster_indices):
    cluster_labels[cluster] = i

score = davies_bouldin_score(cluster_data, cluster_labels)
            
print(f"Number of clusters: {n_clusters}")
    
#cluster_center = clustering_model.cluster_centers_[i]
#cluster_centers = clustering_model.get_means()
cluster_centers = clustering_model.get_medians()

plot_cluster_centers(n_clusters, cluster_centers, times).show()

print(f"BDI: {score}")

{'n_clusters': 9, 'distance_metric': <pyclustering.utils.metric.distance_metric object at 0x7acf4fd0f140>} 1000 <function dtw at 0x7acf60035a80>
Number of clusters: 9


BDI: 0.8087909444389905


kmeans:

{'n_clusters': 14, 'distance_metric': <pyclustering.utils.metric.distance_metric object at 0x7acf4f19fc20>}
Number of clusters: 14
BDI: 0.803127753888182


# Analyize cluster info

For inspiration of quantities to derive look at PRECount paper

In [ ]:
def filter_by_date(dataframe, time_column, date):
    return dataframe[dataframe[time_column].dt.date == date]

def number_courses(dataframe, date):
    courses_of_day = filter_by_date(dataframe, "start_time", date)
    return len(courses_of_day)

def number_exams(dataframe, date):
    courses_of_day = filter_by_date(dataframe, "start_time", date)
    return sum(courses_of_day["exam"] == 1)

def derive_weekend(dataframe, date_column):
    dataframe["weekend"] = dataframe[date_column].apply(lambda x: x.weekday()) >= 5
    return dataframe

national_holidays_2024 = [
    datetime.strptime('01/01/2024', '%m/%d/%Y').date(),  # Neujahr
    datetime.strptime('01/06/2024', '%m/%d/%Y').date(),  # Heilige Drei Könige
    datetime.strptime('04/01/2024', '%m/%d/%Y').date(),  # Ostermontag
    datetime.strptime('05/01/2024', '%m/%d/%Y').date(),  # Staatsfeiertag
    datetime.strptime('05/09/2024', '%m/%d/%Y').date(),  # Christi Himmelfahrt
    datetime.strptime('05/20/2024', '%m/%d/%Y').date(),  # Pfingstmontag
    datetime.strptime('05/30/2024', '%m/%d/%Y').date(),  # Fronleichnam
    datetime.strptime('08/15/2024', '%m/%d/%Y').date(),  # Mariä Himmelfahrt
    datetime.strptime('10/26/2024', '%m/%d/%Y').date(),  # Nationalfeiertag
    datetime.strptime('11/01/2024', '%m/%d/%Y').date(),  # Allerheiligen
    datetime.strptime('12/08/2024', '%m/%d/%Y').date(),  # Mariä Empfängnis
    datetime.strptime('12/25/2024', '%m/%d/%Y').date(),  # Christtag
    datetime.strptime('12/26/2024', '%m/%d/%Y').date(),  # Stefanitag
]

university_holidays_2024 = [
    # 18.05.2024
    datetime.strptime('05/18/2024', '%m/%d/%Y').date(),
    # 21.05.2024
    datetime.strptime('05/21/2024', '%m/%d/%Y').date(),
    # 04.05.2024
    datetime.strptime('05/04/2024', '%m/%d/%Y').date(),
    # 31.05.2024
    datetime.strptime('05/31/2024', '%m/%d/%Y').date(),
]

zwickeltage_2024 = [
    # 10.05.2024
    datetime.strptime('05/10/2024', '%m/%d/%Y').date(),
    # 31.05.2024
    datetime.strptime('05/31/2024', '%m/%d/%Y').date(),
    # 16.08.2024
    datetime.strptime('08/16/2024', '%m/%d/%Y').date(),
]

easter_break_2024 = [
    # easter break: 25.03.2024 - 06.04.2024
    datetime.strptime('03/25/2024', '%m/%d/%Y').date(),
    datetime.strptime('04/06/2024', '%m/%d/%Y').date(),]
summer_break_2024 = [
    # summer break: 1.07.2024 - 30.09.2024
    datetime.strptime('07/29/2024', '%m/%d/%Y').date(),
    datetime.strptime('09/09/2024', '%m/%d/%Y').date(),
]
def calculate_break_dates(break_dates):
    break_days = []
    num_dates = (break_dates[1] - break_dates[0]).days + 1
    for i in range(num_dates):
        break_days.append((break_dates[0] + DateOffset(days=i)).date())
        
    return break_days

easter_break_2024 = calculate_break_dates(easter_break_2024)

summer_break_2024 = calculate_break_dates(summer_break_2024)



def derive_number_exams(dataframe, date_column):
    dataframe["number_exams"] = dataframe[date_column].apply(lambda x: number_exams(course_dates_data, x))
    return dataframe

def derive_number_courses(dataframe, date_column):
    dataframe["number_courses"] = dataframe[date_column].apply(lambda x: number_courses(course_dates_data, x))
    return dataframe

def derive_holiday(dataframe, holiday_dates, date_column, out_column):
    dataframe[out_column] = dataframe[date_column].isin(holiday_dates)
    return dataframe

In [ ]:
def enrich_cluster_infos(cluster_infos):
    cluster_infos = derive_number_courses(cluster_infos, "day")
    cluster_infos = derive_number_exams(cluster_infos, "day")

    cluster_infos = derive_weekend(cluster_infos, "day")

    cluster_infos = derive_holiday(cluster_infos, zwickeltage_2024, "day", "zwickltag")
    cluster_infos = derive_holiday(cluster_infos, national_holidays_2024 + university_holidays_2024, "day", "holiday")

    cluster_infos = derive_holiday(cluster_infos, easter_break_2024 + summer_break_2024, "day", "break")
    
    return cluster_infos

In [ ]:
indices = np.arange(len(occupancy_counts))

for i in range(n_clusters):  
    
    cluster_indices = indices[cluster_labels == i]

    cluster_infos = sample_info[cluster_indices]


    cluster_infos = pd.DataFrame(cluster_infos, index=cluster_indices, columns=["weekday", "weekday_name", "week", "day"])

    cluster_infos = enrich_cluster_infos(cluster_infos)
    
    print("###################")
    print(cluster_infos.to_string())
    print("###################")
    print()

###################
   weekday weekday_name week         day  number_courses  number_exams  weekend  zwickltag  holiday  break
5        5     Saturday   15  2024-04-13               1             0     True      False    False  False
10       3     Thursday   16  2024-04-18               5             0    False      False    False  False
17       3     Thursday   17  2024-04-25               4             0    False      False    False  False
18       4       Friday   17  2024-04-26               1             1    False      False    False  False
24       3     Thursday   18  2024-05-02               4             0    False      False    False  False
28       0       Monday   19  2024-05-06               5             0    False      False    False  False
37       2    Wednesday   20  2024-05-15               5             0    False      False    False  False
38       3     Thursday   20  2024-05-16               4             0    False      False    False  False
44       2    Wed

In [ ]:
# derive the actual courses -> based on start_time 
# E.g.: if a day had a course at 8:30 or not and so on



In [ ]:
# try to cluster the results based on the stuff that happened on that day with decision tree
from sklearn.tree import DecisionTreeClassifier

# clusters are the labels
# cluster_infos are the features



Distance metrics:

* Euclidean
* Correlation Similarity
* Dynamic Time Warp



In [ ]:
# get optimal configuration

from sklearn.cluster import DBSCAN, BisectingKMeans
from sklearn.manifold import TSNE

seeds = range(0,100)

parameters = {
    "general_settings":{},
    "downprojection": False,
    "algorithm":"kmeans",
    "kmeans":{
        "n_clusters": list(range(2, 15)),
    },
    "bisectingkmeans":{
        "n_clusters": list(range(2, 15)),
        "n_init": [1, 5, 10, 20],
    },
    "affinity":{
        "damping": np.linspace(0.5, 1, 10, endpoint=False),
        "affinity": ["euclidean"],
    },
    "dbscan":{
        "eps": np.linspace(0.1, 1, 10),
        "min_samples": list(range(5, 15)),
        "metric": ["euclidean", "manhattan", "correlation"],
        "leaf_size": [2,5,10,30],
    },
    "meanshift":{
        },
    "spectral":{
       "n_clusters": list(range(2, 15)), 
    },
    "agglomerative":{
        "n_clusters": list(range(2, 15)),
        "metric": ["manhattan", "cosine", "l1", "l2"],
        "linkage": ["complete", "average", "single"],
    },
    
    "tsne":{
        "n_components": [3],
        "perplexity": [30],
    }
}


if parameters["algorithm"] == "kmeans":
    algorithm_instance = KMeans
elif parameters["algorithm"] == "bisectingkmeans":
    algorithm_instance = BisectingKMeans
elif parameters["algorithm"] == "affinity":
    algorithm_instance = AffinityPropagation
elif parameters["algorithm"] == "dbscan":
    algorithm_instance = DBSCAN
elif parameters["algorithm"] == "meanshift":
    algorithm_instance = MeanShift
elif parameters["algorithm"] == "spectral":
    algorithm_instance = SpectralClustering
elif parameters["algorithm"] == "agglomerative":
    algorithm_instance = AgglomerativeClustering
else:
    raise ValueError("Algorithm not implemented!")

argmin_list = []
for seed in seeds:
    
    bdi_scores = []
    combination = []
    comb_iterator = ParameterSearch(parameter_dict=parameters[parameters["algorithm"]]).grid_search_iterator(tqdm_bar=True)

    for i, params in enumerate(comb_iterator):
        
        #print(f"################ {n_clusters} ################")
        
        clustering_model = algorithm_instance(**params)
        
        #comb_iterator_tsne = ParameterSearch(parameter_dict=parameters["tsne"]).grid_search_iterator(tqdm_bar=False)
        
        
        #for params_tsne in comb_iterator_tsne:
        #    if parameters["downprojection"]:
        #        samples_embedded = TSNE(**params_tsne).fit_transform(samples)
            
        samples_embedded = samples
        try:
            if params["affinity"] == "precomputed":
                similarities = - pairwise_distances(samples_embedded + 1e-8, metric="manhattan")
                clustering_model.fit(similarities)
            else:
                clustering_model.fit(samples_embedded)
        except:
            clustering_model.fit(samples_embedded + 1e-8)


        #plot_cluster_centers(n_clusters, clustering_model, times[0]).show()
        if len(np.unique(clustering_model.labels_)) == 1:
            score = 1000
        else:
            score = davies_bouldin_score(samples, clustering_model.labels_)

        #print(score)
        bdi_scores.append(np.round(score, 6))
        #combination.append((params, params_tsne))
        combination.append((i, params))

        
    argmin_list.append(np.argmin(bdi_scores))
np.unique(argmin_list, return_counts=True)

100%|██████████| 13/13 [00:00<00:00, 226.09it/s]


(array([ 0,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12]),
 array([ 5,  1,  1,  2,  2,  2,  3,  6,  7, 15, 21, 35]))

### Optimal Parameters

algorithm = KMeans , INVESTIGATE FURTHER -- Read the two papers and implement their approach<br>
include_seeds = True <br>
algo_parameters = {
    "n_clusters":12
}

BisectingKMeans, INVESTIGATE FURTHER -- we need seeds<br>
{'n_clusters': 14, 'n_init': 20}

algorithm = AffinityPropagation -> works well, GRAPHCLUSTERING<br>
{'damping': 0.5, 'affinity': 'euclidean'}

AgglomerativeClustering -> works well <br>
{'n_clusters': 2, 'metric': 'manhattan', 'linkage': 'average'}

Fuzzy C-Means <br>
Implementation is flawed

DBScan <br>
Figure out better parameters


Dimensionality reduction:
* TSNE enhanced Clustering:

No TSNE<br>
{'n_clusters': 14}
Number of clusters: 14
BDI: 0.7672581861245025

* UMAP enahnced Clustering


In [ ]:
combination

[(0, {'n_clusters': 2}),
 (1, {'n_clusters': 3}),
 (2, {'n_clusters': 4}),
 (3, {'n_clusters': 5}),
 (4, {'n_clusters': 6}),
 (5, {'n_clusters': 7}),
 (6, {'n_clusters': 8}),
 (7, {'n_clusters': 9}),
 (8, {'n_clusters': 10}),
 (9, {'n_clusters': 11}),
 (10, {'n_clusters': 12}),
 (11, {'n_clusters': 13}),
 (12, {'n_clusters': 14})]

In [ ]:
# test optimal configuration
print(f"################ {parameters['algorithm']} ################")
optimal_params = combination[10][1]
print(optimal_params)

clustering_model = algorithm_instance(**optimal_params)

try:
    if params["affinity"] == "precomputed":
        similarities = - pairwise_distances(samples + 1e-8, metric="manhattan")
        clustering_model.fit(similarities)
    else:
        clustering_model.fit(samples)
except:
    clustering_model.fit(samples)

try:
    n_clusters = len(clustering_model.cluster_centers_)
except:
    try:
        clustering_model.cluster_centers_ = samples[clustering_model.cluster_centers_indices_]
        n_clusters = len(clustering_model.cluster_centers_indices_)
    except:
        n_clusters = len(np.unique(clustering_model.labels_))
    
    

print(f"Number of clusters: {n_clusters}")

#plot_cluster_centers(n_clusters, clustering_model, times[0]).show()

score = davies_bouldin_score(samples, clustering_model.labels_)
print(f"BDI: {score}")

################ kmeans ################
{'n_clusters': 12}
Number of clusters: 12
BDI: 0.809285789249472


In [ ]:
for i in range(n_clusters):  
    plot_cluster_details(i, clustering_model, samples, times[0], plot_centers=False).show()

################ 0 ################
[ 0  7 14 21]


################ 1 ################
[ 6 12 13 19 20 23 25 26 27 28 31 32 33 34 40 41 42 43 46 47 48 50 52 53
 54 55 60 61 62 64 67 68 69 74 75 76 77]


################ 2 ################
[ 5  8 10 17 18 24 37 38 44 45 58 59 66 72 73 79]


################ 3 ################
[78]


################ 4 ################
[80]


...

################ 7 ################
[22]


################ 8 ################
[ 2  9 16 30 51 57 65]


################ 9 ################
[11 35 49 56 63 70]


################ 10 ################
[3]


################ 11 ################
[ 1 15 36 71]


In [ ]:
indices = np.arange(len(samples))

fig = go.Figure()

for i in range(n_clusters):

    cluster_indices = indices[clustering_model.labels_ == i]
    cluster_center = clustering_model.cluster_centers_[i]
    
    
    cluster_group_list = group_list[cluster_indices]

    print(f"################ {i} ################")
    print(cluster_indices)
    print(cluster_group_list)
    print()
    
    # line plot
    fig.add_trace(
        go.Scatter(
            x=times[0], 
            y=cluster_center, 
            name=f'cluster {i}',
            mode="lines+markers"        
        )
    )
        
fig.show()

################ 0 ################
[ 6 12 13 19 20 23 25 26 27 31 32 33 34 40 41 42 43 46 47 48 52 53 54 55
 60 61 62 67 68 69 74 75 76]
[datetime.date(2024, 4, 14) datetime.date(2024, 4, 20)
 datetime.date(2024, 4, 21) datetime.date(2024, 4, 27)
 datetime.date(2024, 4, 28) datetime.date(2024, 5, 1)
 datetime.date(2024, 5, 3) datetime.date(2024, 5, 4)
 datetime.date(2024, 5, 5) datetime.date(2024, 5, 9)
 datetime.date(2024, 5, 10) datetime.date(2024, 5, 11)
 datetime.date(2024, 5, 12) datetime.date(2024, 5, 18)
 datetime.date(2024, 5, 19) datetime.date(2024, 5, 20)
 datetime.date(2024, 5, 21) datetime.date(2024, 5, 24)
 datetime.date(2024, 5, 25) datetime.date(2024, 5, 26)
 datetime.date(2024, 5, 30) datetime.date(2024, 5, 31)
 datetime.date(2024, 6, 1) datetime.date(2024, 6, 2)
 datetime.date(2024, 6, 7) datetime.date(2024, 6, 8)
 datetime.date(2024, 6, 9) datetime.date(2024, 6, 14)
 datetime.date(2024, 6, 15) datetime.date(2024, 6, 16)
 datetime.date(2024, 6, 21) datetime.date(2024,

## 2. Train and Test split

In [ ]:
data_filterd = filter_room_id(data, 0)
data_filterd = filter_event_type(data_filterd, 1)


#train, test = train_test_split(samples, test_size=0.1, random_state=42)

#train = np.array(train)
#test = np.array(test)

#kf = KFold(n_splits=10)

#for train_index, valid_index in kf.split(train):
    
#    train_fold = train[train_index]
#    valid_fold = train[valid_index]
    
#    break

## 3. Occupancy Forecasting

In [ ]:
# check out some of the papers in the review
# start with some traditional methods

## 4. Count Forecasting
### Try more general approach if 1. is too hard, gather literature on count forecasting